In [24]:
import spacy
nlp = spacy.load("en_core_web_lg")

import numpy as np
import re
import importlib
import time
from typing import Optional

import ogree_wiki as wiki
importlib.reload(wiki)

<module 'ogree_wiki' from 'c:\\Users\\thoxy\\OneDrive\\Documents\\IMT-PJENT\\OGrEE_NLP\\ogree_wiki.py'>

In [25]:
ACTIONS_DEFAULT = {
                    "ACTION_POSITIVE" : ["make","build","put","place"],
                    "ACTION_NEGATIVE" : ["remove", "delete"], 
                    "ALTERATION" : ["modify", "change","move","set","rename","rotate"]
                    }

ACTIONS_CLI = {
                "ACTION_POSITIVE" : "+",
                "ACTION_NEGATIVE" : "-"
                }

SIMILARITY_PARAMETER = 0.5

KEY_WORDS_DICT = {
            "name" : ["name","called"],
            "position" : ["position","at","located"],
            "rotation" : ["rotation","turned","degree"],
            "size" : ["size","dimensions"],
            "template" : ["template"],
            "axisOrientation" : ["axis", "orientation"],
            "floorUnit" : ["floor","unit"],
            "slot" : ["slot"]
            }


In [26]:
def findIndexAction(processed_entry : type(nlp("")), indexAction : int, list_index_entities : list) -> int :
    # identify relations for each entity recognized
    currentIndexes = list_index_entities[::]
    counter = 0
    while counter < 3 :
        for i in range(len(list_index_entities)) :
            currentWord = processed_entry[currentIndexes[i]].head
            currentIndexes[i] = list(processed_entry).index(currentWord)
        counter += 1

    currentIndexesBool = [index == indexAction for index in currentIndexes]
    if sum(currentIndexesBool) >= 2 :
        currentIndexDistance = [(abs(indexAction - list_index_entities[i]),list_index_entities[i]) for i in range(len(currentIndexes)) if currentIndexesBool[i] == True]
        return min(currentIndexDistance)[1]
    elif sum(currentIndexesBool) == 1:
        return list_index_entities[currentIndexesBool.index(True)]
    else :
        raise Exception("Interpretation issue : no relation found, please try again")


In [27]:
def findIndexMainSubject(processed_entry : type(nlp("")), indexAction : int, dictioIndexKeyWords : dict) -> int :

    counter = 0
    currentIndexes = {index:index for index,parameter in dictioIndexKeyWords.items() if parameter not in ACTIONS_DEFAULT.keys()}
    currentWords = {index:processed_entry[index] for index,parameter in dictioIndexKeyWords.items() if parameter not in ACTIONS_DEFAULT.keys()}
    while (not indexAction in currentIndexes.values()) and counter < 3 :
        currentWords = {originIndex : processed_entry[currentIndex].head for originIndex,currentIndex in currentIndexes.items()}
        currentIndexes = {originIndex : list(processed_entry).index(currentWords[originIndex]) for originIndex,_ in currentIndexes.items()}
        counter += 1

    if counter == 3 :
        raise Exception("Did not find the main action")
    
    if list(currentIndexes.values()).count(indexAction) != 1 :
        # if many options remain, we only consider those at the right of the verb
        listIndexesRemaining = [originIndex for originIndex,currentIndex in currentIndexes.items() if currentIndex == indexAction and originIndex > indexAction]
        return listIndexesRemaining[0]
    else :
        return [originIndex for originIndex,currentIndex in currentIndexes.items() if currentIndex == indexAction][0]


In [28]:
def name(processed_entry : type(nlp("")), dictioEntities : dict, indexesMain : list) -> int :

    def findClose(processed_entry : type(nlp("")), index : int) -> (int|None) :
        if index +1 <= len(processed_entry)-1 :
            if str(processed_entry[index+1]).isupper() or processed_entry[index+1].pos_ == "PROPN" :
                return index+1
        if 0 <= index -1 :
            if str(processed_entry[index-1]).isupper() or processed_entry[index-1].pos_ == "PROPN" :
                return index-1
        return None
    

    def findAttachedEntity(processed_entry : type(nlp("")), index : int) -> (int|None) : 
        counter = 0
        isFound = False
        currentIndex = index
        currentWord = processed_entry[currentIndex]
        while (not isFound) and counter < 3 :
            currentWord = processed_entry[currentIndex].head
            currentIndex = list(processed_entry).index(currentWord)
            if str(currentWord) in dictioEntities.values() :
                return currentIndex
            if currentWord.pos_ == "VERB" and currentIndex == indexesMain[1] : 
                return indexesMain[0]
            counter += 1
        return None
    
    EXPLICIT =  KEY_WORDS_DICT["name"]
    IMPLICIT = ["current","main"]

    dictioEntityNames = {}
        
    for index,entity in dictioEntities.items() :
    # if the name if right beside the entity
        resultIndex = findClose(processed_entry, index)
        if resultIndex != None :
            dictioEntityNames[index] = resultIndex

    if len(dictioEntityNames) < len(dictioEntities) : # if not all names found

        for index,token in enumerate(processed_entry) : # look for keyword

            if str(processed_entry[index]).isupper() or processed_entry[index].pos_ == "PROPN" :
                indexAttachedEntity = findAttachedEntity(processed_entry,index)
                if not (indexAttachedEntity in dictioEntityNames or indexAttachedEntity == None) :
                    dictioEntityNames[indexAttachedEntity] = index
            
            # if the token is a synonym of called
            if sum([token.similarity(nlp(word)[0]) > SIMILARITY_PARAMETER for word in EXPLICIT]) >= 1 :
                resultIndex = findClose(processed_entry, index)
                if resultIndex != None :
                    indexAttachedEntity = findAttachedEntity(processed_entry,index)
                    if not (indexAttachedEntity in dictioEntityNames or indexAttachedEntity == None) :
                        dictioEntityNames[indexAttachedEntity] = resultIndex


    if len(dictioEntityNames) < len(dictioEntities) : # if still not all names found
        pass
    # TODO : get current names

    # TODO : check for implicit words now
    
    return dictioEntityNames

In [29]:
def position(processed_entry : type(nlp("")), index : int, lastKeyWordIndex : int, nextKeyWordIndex : int, entity : str) -> Optional[list] :
    # search for keywords to the right and left to split whole sentence into parts where to look for param entries
    position_list = re.findall("[-]*[0-9]+[.]*[0-9]*", " ".join([str(token) for token in processed_entry[index+1:nextKeyWordIndex]]))
    if not (len(position_list) == 2 or (len(position_list) == 3 and entity == "rack")):
        position_list_left = re.findall("[-]*[0-9]+[.]*[0-9]*", " ".join([str(token) for token in processed_entry[lastKeyWordIndex:index]]))
        if not (len(position_list) == 2 or (len(position_list) == 3 and entity == "rack")) :
            raise Exception("Wrong location format entered")
        else :
            position_list = position_list_left
    result = [float(coord) for coord in position_list] if position_list else None
    return result

In [30]:
def rotation(processed_entry : type(nlp("")), index : int, lastKeyWordIndex : int, nextKeyWordIndex : int, entity : str) -> Optional[float] :
    next_words = processed_entry[index+1:nextKeyWordIndex]
    last_words = processed_entry[lastKeyWordIndex:index]
    
    isRotationNegative = False
    rotation_list = re.findall("[-]*[0-9]+[.]*[0-9]*", " ".join([str(token) for token in next_words]))
    isRotationNegative = re.search("counter.*clockwise", "".join([str(token) for token in next_words]))
    if not len(rotation_list) == 1 :
        rotation_list_left = re.findall("[-]*[0-9]+[.]*[0-9]*", " ".join([str(token) for token in last_words]))
        isRotationNegative = re.search("counter.*clockwise", "".join([str(token) for token in next_words]))
        if not len(rotation_list_left) == 1 :
            raise Exception("Wrong rotation format entered")
        else :
            rotation_list = rotation_list_left
    rotationFinal = float(rotation_list[0]) if rotation_list else None
    rotationFinal = -rotationFinal if isRotationNegative else rotationFinal
    return rotationFinal

In [31]:
# TODO : the similarity func is very long, we must shorten the process time or find another way

def main() -> str :
    FINAL_INSTRUCTION = ""

    ENTITIES_FULL_NAME = {"entity" : list(wiki.ENTITIES.keys())}

    # TODO : add already existing entity names
    KEY_WORDS_ALL = {**ENTITIES_FULL_NAME,  **KEY_WORDS_DICT}
    KEY_WORDS_ENTRY = {}

    natural_entry = input("Enter a prompt. Please follow the instructions given.\n")
    processed_entry = nlp(natural_entry)

    # we detect key words in the sentence given and put them into KEY_WORDS_ENTRY
    for index,token in enumerate(processed_entry) :
        matching_list = []
        if token.pos_ == "VERB" :
            for parameter in ACTIONS_DEFAULT.keys() :
                similarity = max([token.similarity(nlp(word)[0]) for word in ACTIONS_DEFAULT[parameter]])
                matching_list.append((similarity,parameter))
        elif token.pos_ == "NOUN":
            for parameter in KEY_WORDS_ALL.keys() : 
                similarity = max([token.similarity(nlp(word)[0]) for word in KEY_WORDS_ALL[parameter]])
                matching_list.append((similarity,parameter))
        else :
            continue
        match = max(matching_list)
        if match[0] > SIMILARITY_PARAMETER :
            KEY_WORDS_ENTRY[index] = match[1]

    print(KEY_WORDS_ENTRY)

    # test detection
    list_key_param = list(KEY_WORDS_ENTRY.values())
    count_action = 0
    for action_type in ACTIONS_DEFAULT.keys() :
        count_action += list_key_param.count(action_type)

    if count_action != 1 :
        raise Exception("Action not detected")
    else :
        global INDEX_ACTION
        global INDEX_MAIN_SUBJECT
        INDEX_ACTION = [index for index,keyword in KEY_WORDS_ENTRY.items() if keyword in ACTIONS_DEFAULT.keys()][0]
        INDEX_MAIN_SUBJECT = findIndexMainSubject(processed_entry, INDEX_ACTION, KEY_WORDS_ENTRY)  
    print("index main : ",INDEX_MAIN_SUBJECT)

    dictEntities = {index : str(processed_entry[index]) for index,keyword in KEY_WORDS_ENTRY.items() if keyword == "entity"}
    print(dictEntities)
    dictioEntityNames = name(processed_entry, dictEntities, [INDEX_MAIN_SUBJECT, INDEX_ACTION])
    print("names : ",dictioEntityNames)

    # TODO : match cases (main = entity/parameter, verb = +/alteration...)
    if KEY_WORDS_ENTRY[INDEX_MAIN_SUBJECT] == "entity" :

        if KEY_WORDS_ENTRY[INDEX_ACTION] == "ACTION_POSITIVE" : 
            # we do the processes related to each parameter
            dictioParameters = wiki.makeDictParam(str(processed_entry[INDEX_MAIN_SUBJECT]))
            for index,parameter in KEY_WORDS_ENTRY.items() :
                if (not KEY_WORDS_ENTRY[index] in KEY_WORDS_DICT.keys()) or bool(dictioParameters[parameter]) == True :
                    continue
                print("here")
                parameterValue = locals()[parameter]()
                dictioParameters[parameter] = parameterValue

        if KEY_WORDS_ENTRY[INDEX_ACTION] == "ACTION_NEGATIVE" :
            # TODO : only check if the entity is well detected
            pass

        if KEY_WORDS_ENTRY[INDEX_ACTION] == "ALTERATION" :
            # TODO : understand what are the changes and its consequences
            pass

    if KEY_WORDS_ENTRY[INDEX_MAIN_SUBJECT] in KEY_WORDS_DICT.keys() :
        # TODO : seek the value to be changed
        pass


    
    # if seeking the name for the main entity, pass the indexaction as parameter
    # if no name found, check the type of action : if +, a name is needed, otherwise not necessarily

    # check if parameters were not given

    return KEY_WORDS_ENTRY


In [32]:
text = ""
main()

{0: 'ACTION_POSITIVE', 2: 'entity'}
index main :  2
{2: 'building'}
names :  {}


{0: 'ACTION_POSITIVE', 2: 'entity'}

In [33]:
nlp("name")[0].similarity(nlp("rename")[0])
# [nlp("current")[0].similarity(nlp(word)[0]) for word in ["axis", "orientation"]]

0.6389259696006775

In [34]:
bool(None)

False